In [68]:
import warnings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
warnings.filterwarnings("ignore")

In [69]:
import glob
import os
warnings.filterwarnings("ignore")
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

def load_pdf_file(data):
    """
    Load all PDF files from the given directory using DirectoryLoader and PyPDFLoader.
    
    Args:
        data (str): Path to the folder containing PDF files.
    
    Returns:
        list: List of langchain Document objects.
    """
    if not os.path.exists(data):
        raise FileNotFoundError(f"Directory not found: '{data}'")
        
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs


In [70]:
docs = load_pdf_file('.')  # '.' means current directory
print(f"Loaded {len(docs)} documents.")

Ignoring wrong pointing object 1304 0 (offset 0)
Ignoring wrong pointing object 1306 0 (offset 0)
Ignoring wrong pointing object 1458 0 (offset 0)
Ignoring wrong pointing object 1502 0 (offset 0)
Ignoring wrong pointing object 1812 0 (offset 0)
Ignoring wrong pointing object 1815 0 (offset 0)
Ignoring wrong pointing object 3289 0 (offset 0)
Ignoring wrong pointing object 3436 0 (offset 0)
Ignoring wrong pointing object 4032 0 (offset 0)
Ignoring wrong pointing object 4129 0 (offset 0)
Ignoring wrong pointing object 4153 0 (offset 0)
Ignoring wrong pointing object 4185 0 (offset 0)
Ignoring wrong pointing object 4769 0 (offset 0)
Ignoring wrong pointing object 4806 0 (offset 0)
Ignoring wrong pointing object 4832 0 (offset 0)
Ignoring wrong pointing object 4888 0 (offset 0)
Ignoring wrong pointing object 5030 0 (offset 0)
Ignoring wrong pointing object 5104 0 (offset 0)
Ignoring wrong pointing object 5114 0 (offset 0)
Ignoring wrong pointing object 5286 0 (offset 0)
Ignoring wrong point

Loaded 1784 documents.


In [71]:
def text_split(extr_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extr_data)
    return text_chunks

In [72]:
text_chunks = text_split(docs)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  11833


In [73]:
# Download the Embedding model from Hugging Face
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [74]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = download_hugging_face_embedding()


In [75]:
#os.chdir("C:\\Users\\ENG.YAHYA\\Desktop\\projects\\FinTutor-Chatbot-Project\\model")

In [76]:
import sentence_transformers
print(sentence_transformers.__version__)

5.0.0


In [77]:
#query_result = embeddings.embed_query("Hello world")
#print(query_result)
#len(query_result)


In [78]:
print(embeddings.model_name)

sentence-transformers/all-MiniLM-L6-v2


In [79]:
from dotenv import load_dotenv
load_dotenv()

True

In [80]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPEN_ROUTER_API_KEY")



In [82]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
import os 

pc = pinecone(api_key=pinecone_api_key)

index_name = "fintutor"

pc.create_index(
    index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws",region="us-east-1")
)



{
    "name": "fintutor",
    "metric": "cosine",
    "host": "fintutor-bcouli4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [83]:
import openai

openai_api_key = "OPEN_ROUTER_API_KEY"
openai_api_base = "https://openrouter.ai/api/v1"


In [84]:
import os
pinecone_api_key = os.getenv("PINECONE_API_KEY")
openai_api_key = os.getenv("OPEN_ROUTER_API_KEY")


In [85]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name= index_name,
    embedding=embeddings
    )

In [86]:
## Load the Existing index
from langchain_pinecone import PineconeVectorStore
## Embed the each chunk and upsert the embeddings into your pincone index.
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [87]:
docsearch

In [89]:
retriver = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [90]:
retrived_docs = retriver.invoke("what is the bond")


In [91]:
retrived_docs

[Document(id='e834ed22-18bf-4ac4-99a5-f93e78a9f3c2', metadata={'creationdate': '2022-03-16T14:06:07-05:00', 'creator': 'PyPDF', 'moddate': '2022-03-17T09:02:07-05:00', 'page': 326.0, 'page_label': '327', 'producer': 'Prince 14.2 (www.princexml.com)', 'source': 'Principle_of_finance.pdf', 'title': 'Principles of Finance', 'total_pages': 643.0}, page_content='Summary\n10.1 Characteristics of Bonds\nBonds are typically a basic form of investment that entails a straightforward financial agreement between\nissuer and purchaser. There are three primary categories of bonds: government bonds, corporate bonds, and\nconvertible bonds. These different types of bond vary depending on their issuer, length until maturity, interest\nrate, and risk.\n10.2 Bond Valuation'),
 Document(id='9e041de7-408d-46e2-b635-0c2953facc39', metadata={'creationdate': '2022-03-16T14:06:07-05:00', 'creator': 'PyPDF', 'moddate': '2022-03-17T09:02:07-05:00', 'page': 326.0, 'page_label': '327', 'producer': 'Prince 14.2 (ww

In [102]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    openai_api_key = openai_api_key,
    openai_api_base = openai_api_base,
    model="mistralai/mistral-7b-instruct:free", 
    temperature=0.4,
    max_tokens=500,
)



In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a helpful finance tutor. "
    "Use the following pieces of retrieved context to answer the student's question. "
    "If you don't know the answer, just say that you don't know. "
    "Use clear, simple language and keep your answer concise (maximum three sentences). "
    "If possible, provide a brief example or explanation."
    "\n\n"
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]

)



In [109]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)


In [110]:
response = rag_chain.invoke({"input": "What is Finance?"})
print(response["answer"])

 Finance is the study of managing, moving, and raising money, with a focus on the trade-off between risk and expected return. It is divided into three main areas: business finance, investments, and financial markets and institutions. In an organization, finance plays a crucial role in implementing policy objectives, monitoring results, and responding to strategic and tactical changes. It acts as a linchpin that connects and directs various parts of a business or organization.
